**Question 1**

6 TSQL Queries

In [8]:
USE TiorGames;

Commands completed successfully.

Total execution time: 00:00:00.181

In [9]:
SELECT 
    Rank() Over(Order by Sum(SpectatorsNumber + VIPSpectatorsNumber) Desc) As Ranking, 
    StadiumName as [Name of Stadium], Country, EventName as [Event Type], 
    Sum(SpectatorsNumber) as [Number of Spectators], 
    Sum(VIPSpectatorsNumber) as [Number of VIP Spectators], 
    Sum(SpectatorsNumber + VIPSpectatorsNumber) as [Total Number of Spectators]
FROM GameFact gf
Inner JOIN EventFact ef ON ef.EventID = gf.EventID
INNER JOIN EventDim ed ON ed.EventID = gf.EventID
INNER JOIN StadiumDim sd ON sd.StadiumID = gf.StadiumId
INNER JOIN LocationDim ld on ld.LocationID = sd.StadiumLocationID
GROUP BY StadiumName, EventName, Country;

(10 rows affected)

Total execution time: 00:00:00.186

Ranking,Name of Stadium,Country,Event Type,Number of Spectators,Number of VIP Spectators,Total Number of Spectators
1,Copper Box Arena,United Kingdom,msi,2676564,154440,2831004
2,KeyArena,United States,msi,2416040,147345,2563385
3,Commerzbank Arena,Germany,msi,2219360,136770,2356130
4,San Jose SAP Centre,United States,Worlds,2120712,125367,2246079
5,Wembley Arena,United Kingdom,msi,2028180,133056,2161236
6,Spodek Arena,Poland,Worlds,1875200,108000,1983200
7,Staples Centre,United States,msi,1812800,114400,1927200
8,Sang-am World Cup Stadium,"Korea, South",Worlds,1622313,99792,1722105
9,CSKA Arena,Russia,Worlds,1456154,88830,1544984
10,Royal Arena,Denmark,Worlds,1044555,58100,1102655


In [15]:
SELECT
    MerchandiseType AS [Merchandise Type], 
    Sum(MerchandiseStocked) As [Total Stocked], 
    Sum(MerchandiseSold) As [Total Sold], 
    Concat(ROUND(CAST(Sum(MerchandiseSold) AS FLOAT) / Sum(MerchandiseStocked) * 100, 2), ' %')  AS [Percentage Sold]
From EventFact ef
Inner Join MerchandiseDim md on md.MerchandiseID = ef.MerchandiseID
GROUP By MerchandiseType
Order By [Percentage Sold] DESC

(8 rows affected)

Total execution time: 00:00:00.215

Merchandise Type,Total Stocked,Total Sold,Percentage Sold
figures,325975,322399,98.9 %
art and book,286747,273386,95.34 %
accessories,379966,341998,90.01 %
plush,244565,209469,85.65 %
clothing,448696,383316,85.43 %
statues,436092,357701,82.02 %
pins,593752,479014,80.68 %
board games,321199,242152,75.39 %


In [11]:
SELECT 
    EventYear As Year, 
    Concat('$ ', (Sum(MerchandiseSoldPND) - Sum(MerchandiseStockedPND) - Sum(MerchandiseRefundedPND))) AS [Earnings From Merchandise Sale], 
    Concat('$ ', (Sum(TicketsSoldPND) - Sum(TicketsRefundedPND))) As [Earnings From Tickets Sale]
From EventFact ef
Inner Join RefundFact rf on ef.TicketID = rf.TicketID
Inner join EventDim td on ef.EventID = td.EventID
GROUP By EventYear
Order BY EventYear DESC;

(6 rows affected)

Total execution time: 00:00:00.218

Year,Earnings From Merchandise Sale,Earnings From Tickets Sale
2021,$ -61016970,$ 46492999
2020,$ -58660782,$ 39025533
2019,$ -122592552,$ 79449463
2018,$ -122817410,$ 95174578
2017,$ -132778462,$ 87824513
2016,$ -124522801,$ 85746916


In [12]:
WITH RankedPromotions AS (
    SELECT 
        PromotionType AS [Type of Promotion],
        ed.EventName AS [Event Name],
        pd.PromotionDuration AS [Promotion Duration],
        md.MarketeerName AS [Marketeer],
        PromotionCost AS [Promotion Cost Price],
        PromotionRevenue - PromotionCost AS [Promotion Profit],
        ROW_NUMBER() OVER (
            PARTITION BY ed.EventName
            ORDER BY (PromotionRevenue - PromotionCost) DESC, pd.PromotionDuration ASC
        ) AS ranking
    FROM EventFact ef
    INNER JOIN PromotionDim pd ON pd.PromotionID = ef.PromotionID
    INNER JOIN EventDim ed ON ed.EventID = ef.EventID
    INNER JOIN MarketeerDim md ON pd.MarketeerID = md.MarketeerID
)
SELECT *
FROM RankedPromotions
WHERE ranking <= 10
ORDER BY [Event Name], [Promotion Profit] DESC, [Promotion Duration] ASC;

(20 rows affected)

Total execution time: 00:00:00.189

Type of Promotion,Event Name,Promotion Duration,Marketeer,Promotion Cost Price,Promotion Profit,ranking
General Advertising,msi,45,Mynte,2601.00,41313.00,1
Digital Promotions,msi,28,Flashset,2545.00,40732.00,2
General Advertising,msi,70,Gigashots,2849.00,40034.00,3
Digital Promotions,msi,3,Zooxo,3386.00,39856.00,4
Public Relations,msi,85,Rhyzio,4278.00,39340.00,5
General Advertising,msi,83,Avaveo,3953.00,39108.00,6
Sponsorships,msi,7,Tagcat,3000.00,39006.00,7
Sponsorships,msi,18,Avaveo,2736.00,39000.00,8
Sales Promotion,msi,85,Shufflebeat,4876.00,38791.00,9
Public Relations,msi,25,Linkbridge,3506.00,38672.00,10


In [13]:
SELECT 
    HighlightType as [Highlight Type], 
    Count(gf.HighlightID) as [Number of Occurance]
FROM GameFact gf
INNER JOIN HighlightDim hd on hd.HighlightID = gf.HighlightID
GROUP BY HighlightType
ORDER BY [Number of Occurance] DESC;

(10 rows affected)

Total execution time: 00:00:00.186

Highlight Type,Number of Occurance
Legendary!,12
pentakill,8
Godlike!,8
Dominating!,7
quadrakill,7
doublekill,6
Killing spree!,6
triplekill,5
Unstoppable!,3
Rampage!,3


In [14]:
SELECT TOP 10
    CoachName AS [Coach Name],
    MAX(CoachYearsOfExperience) AS [Total Years of Experience],
    STRING_AGG(CoachPosition, ', ') AS [Positions],
    SUM(AwardValueInPND) AS [Total Winnings]
FROM CoachDim cd
INNER JOIN ClubCoachDim ccd ON ccd.CoachID = cd.CoachID
INNER JOIN AwardDim ad ON ad.AwardID = cd.CoachHigherAwardID
GROUP BY CoachName
ORDER BY [Total Winnings] DESC, [Total Years of Experience] DESC;

(10 rows affected)

Total execution time: 00:00:00.184

Coach Name,Total Years of Experience,Positions,Total Winnings
Hanson Coxwell,8,draft coach,2418750
Maura Giorgetti,7,draft coach,2418750
Aldus Keysel,4,"draft coach, sports psychologist",1741500
Celinda Anten,10,"sports psychologist, strategic coach",1335682
Raeann Cuss,3,"head coach, sports psychologist, draft coach",1140750
Florella Dedden,0,"head coach, strategic coach, head coach",1140750
Lanny Derks,10,"strategic coach, sports psychologist, head coach",801753
Wenona Krink,8,"sports psychologist, head coach",800000
Chrissie Tarr,9,"draft coach, head coach, draft coach",774000
Jacqui Dunkerly,4,"head coach, head coach, draft coach",774000
